In [2]:
import pyads
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import tensorflow
import keras
import numpy as np
import os


plc_buffer_name = 'Main.profiler_buffer'
plc_numlayers_name = 'Main.FB_NN.nn.num_layers'
plc_numneurons_name = 'Main.FB_NN.nn.layers[[index]].num_neurons'
plc_MeasureFinished_name = 'Main.MeasureFinished'
plc_output_name = 'Main.out'
plc_input_name = 'Main.input'

num_layers = [6, 5, 4, 3]
num_neurons = [500, 400, 300, 200, 100, 10]
scaling_ind = [1, 1e2, 1e4, 1e6]

In [34]:
python_model = keras.saving.load_model('test_model.keras')
np.random.seed(2024)
measurement = []
for scaling in scaling_ind:
    rand_input = (np.random.rand(100,5)-0.5)*scaling
    with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
        memory = plc.read_by_name(plc_output_name, pyads.PLCTYPE_LREAL)
    for i in range(rand_input.shape[0]):
        
        input = rand_input[i,:]
        python_input = input.reshape(1,5)
        python_output = python_model.predict(python_input, verbose=0)
        with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
            plc.write_by_name(plc_input_name, input.tolist() ,pyads.PLCTYPE_LREAL*5)
        while(1):
            with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
                rtnn_output = plc.read_by_name(plc_output_name, pyads.PLCTYPE_LREAL)
            if memory!=rtnn_output:
                memory = rtnn_output
                break
        dif = python_output[0,0]-rtnn_output
        measurement.append([python_output[0,0],rtnn_output, dif, abs((dif/python_output[0,0])*100), scaling])

df_abv = pd.DataFrame(measurement, columns=['python_output', 'rtnn_output', 'difference', 'percent %', 'scaling'])


In [7]:
fig = px.histogram(df_abv, x="percent %", color='scaling')
fig.show()

In [35]:
fig = px.box(df_abv, x="percent %", color='scaling')
fig.show()

In [36]:
folder = 'orig_measurement_df'
filepath = os.path.join(folder,'precision_relu_100_1h.pkl')
df_abv.to_pickle(filepath)